In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11053
2,Huelva,Confirmados PDIA 14 días,830
3,Huelva,Tasa PDIA 14 días,"162,50834568462025"
4,Huelva,Confirmados PDIA 7 días,377
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,0
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,102


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11053.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3123.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1228 personas en los últimos 7 días 

Un positivo PCR cada 582 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11053.0,377.0,830.0,510743.0,73.814032,162.508346,124.0
Huelva-Costa,6240.0,262.0,540.0,288115.0,90.935911,187.425160,67.0
Condado-Campiña,3404.0,105.0,264.0,155057.0,67.717033,170.259969,54.0
Huelva (capital),3123.0,117.0,247.0,143663.0,81.440594,171.930142,32.0
Palma del Condado (La),539.0,20.0,64.0,10761.0,185.856333,594.740266,11.0
Cartaya,411.0,40.0,63.0,19974.0,200.260338,315.410033,11.0
Palos de la Frontera,249.0,20.0,36.0,11289.0,177.163611,318.894499,10.0
Isla Cristina,446.0,41.0,71.0,21264.0,192.814146,333.897667,10.0
Bonares,64.0,16.0,22.0,6058.0,264.113569,363.156157,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,104.0,11.0,22.0,3371.0,326.312667,652.625334,5.0
Palma del Condado (La),539.0,20.0,64.0,10761.0,185.856333,594.740266,11.0
Jabugo,46.0,6.0,11.0,2250.0,266.666667,488.888889,0.0
Bonares,64.0,16.0,22.0,6058.0,264.113569,363.156157,8.0
Isla Cristina,446.0,41.0,71.0,21264.0,192.814146,333.897667,10.0
Palos de la Frontera,249.0,20.0,36.0,11289.0,177.163611,318.894499,10.0
Cartaya,411.0,40.0,63.0,19974.0,200.260338,315.410033,11.0
Escacena del Campo,97.0,3.0,6.0,2284.0,131.348511,262.697023,0.0
Aljaraque,438.0,21.0,54.0,21260.0,98.777046,253.998119,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,250.0,1.0,17.0,14272.0,7.006726,119.114350,1.0,0.058824
San Juan del Puerto,154.0,1.0,16.0,9300.0,10.752688,172.043011,1.0,0.062500
Paterna del Campo,68.0,2.0,8.0,3509.0,56.996295,227.985181,0.0,0.250000
Palma del Condado (La),539.0,20.0,64.0,10761.0,185.856333,594.740266,11.0,0.312500
Trigueros,133.0,2.0,6.0,7713.0,25.930248,77.790743,1.0,0.333333
Punta Umbría,235.0,3.0,9.0,15242.0,19.682456,59.047369,1.0,0.333333
Gibraleón,350.0,1.0,3.0,12607.0,7.932101,23.796304,0.0,0.333333
Moguer,378.0,15.0,41.0,22088.0,67.910177,185.621152,8.0,0.365854
Aljaraque,438.0,21.0,54.0,21260.0,98.777046,253.998119,6.0,0.388889
